# Components

> Components that are the building blocks to the UI

## Imports

In [1]:
#| default_exp components

In [2]:
#| hide
#| export
from fasthtml.common import *
from fasthtml.svg import Svg
from enum import Enum, EnumType
from fasthtml.components import Uk_select,Uk_input_tag
from functools import partial

In [3]:
from nbdev.showdoc import show_doc
from IPython.display import HTML

## Utils

### Tests

In [4]:
from fastcore.test import *

In [5]:
def test_xml_eq(a, b):
    "Tests if if 2 FT components generate the same xml"
    test_eq(*map(lambda x: x if isinstance(x,str) else to_xml(x), (a, b)))

### Other

In [109]:
#| export
# need a better name, stringify might be too general for what it does 
def stringify(o # String, Tuple, or Enum options we want stringified
             ): # String that can be passed FT comp args (such as `cls=`)
    "Converts input types into strings that can be passed to FT components"  
    if is_listy(o): return ' '.join(map(str,o)) if o else ""
    return o.__str__()

In [107]:
show_doc(stringify)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/components.py#L18){target="_blank" style="float:right; font-size:smaller"}

### stringify

>      stringify (o)

*Converts input types into strings that can be passed to FT components*

|    | **Details** |
| -- | ----------- |
| o | String, Tuple, or Enum options we want stringified |

In [7]:
assert stringify('abc') == 'abc'
assert stringify(('abc','def')) == 'abc def'
assert 'uk-input ' + stringify(()) == 'uk-input '
assert 'uk-input ' + stringify("") == 'uk-input '

In [8]:
#| export
class VEnum(Enum):
    def __add__(self, other):
        "Add other enums, listy, or strings"
        return stringify((self, other))

    def __radd__(self, other):
        "Add other enums, listy, or strings"
        return stringify((other, self))
    
    def __str__(self):
        "Stringifies with uk-{attr}-{value} format"
        base = self.__class__.__name__       
        if isinstance(self.__class__, EnumType):
            base = base.lstrip('Uk').rstrip('T')
        return f"uk-{base.lower()}-{self.value}".strip('-')

## Enums

### Theme

You can select a theme color to and get all the headers.  This brings in the most standard grouping:
+ Tailwind: Optional, but typically FrankenUI is used in conjunction with tailwind
+ Uikit: Many things are styled with uikit
+ UIkit icons: Gives many icons OOTB, and is nice to have handy.
+ Franken WC: Neccesarry for some of the (minimal) interactivity components have

In [9]:
#| export
class Theme(Enum):
    slate = "slate"
    stone = "stone"
    gray = "gray"
    neutral = "neutral"
    red = "red"
    rose = "rose"
    orange = "orange"
    green = "green"
    blue = "blue"
    yellow = "yellow"
    violet = "violet"
    zinc = "zinc"

    def headers(self):
        js = (Script(src="https://cdn.tailwindcss.com"),
              Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"),
              Script(src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"),
              Script(type="module", src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js")
              )
        _url = f"https://unpkg.com/franken-wc@0.0.6/dist/css/{self.value}.min.css"
        return (*js, Link(rel="stylesheet", href=_url))

In [121]:
hdrs = Theme.blue.headers()
app = FastHTML(hdrs=hdrs)
hdrs

(script(('',),{'src': 'https://cdn.tailwindcss.com'}),
 script(('',),{'src': 'https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js'}),
 script(('',),{'src': 'https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js'}),
 script(('',),{'type': 'module', 'src': 'https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js'}),
 link((),{'rel': 'stylesheet', 'href': 'https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css'}))

### Style

#### `B` (Base)

Base enums all will end with `B`.  These are designed to make classes discoverable in python.  These should be generally a 1:1 mapping to a class.

These are the lowest level class selection options without dropping into strings.  We don't need to cover everything, but we want most useful things in here.

In [11]:
#| export
class TextB(Enum):
    sz_xsmall = 'text-xs'
    sz_small = 'text-sm'
    sz_medium = 'text-base'
    sz_large = 'text-lg'
    cl_muted = 'uk-text-muted'
    
    wt_light = 'font-light'
    wt_normal = 'font-normal'
    wt_medium = 'font-medium'
    wt_bold = 'font-bold'
# font-thin	font-weight: 100;
# font-extralight	font-weight: 200;
# font-light	font-weight: 300;
# font-normal	font-weight: 400;
# 
# font-semibold	font-weight: 600;
# font-bold	font-weight: 700;
# font-extrabold	font-weight: 800;
# font-black    
    
    
# font-medium text-sm
    def __str__(self):
        return self.value

### `T` (Transform)

Transform enums all will end with `T`.  These are designed to make useful class groups discoverable in python.

Each option should be a group of enums that are useful in many cases.  For example `muted_sm` is very commonly used as a subheader below a card title and looks great.  These should be high level things that users can generally just pick from to have a decent default without much thought.

In [12]:
#| export
class TextT(Enum):
    muted_xs = TextB.sz_xsmall, TextB.cl_muted 
    muted_sm = TextB.sz_small, TextB.cl_muted # Text below card headings
    muted_med = TextB.sz_medium, TextB.cl_muted 
    muted_lg = TextB.sz_large, TextB.cl_muted 
    medium_sm = TextB.sz_small, TextB.wt_medium
    medium_xs = TextB.sz_xsmall, TextB.wt_medium

    def __str__(self):
        if is_listy(self.value): return ' '.join(map(str,self.value))
        return self.value

In [63]:
type(TextT.muted_sm)

<enum 'TextT'>

## Base Components

### Inputs

In [90]:
#| export
from typing import Union, Tuple, Optional
from fastcore.all import L
# from fasthtml.common import FT

def UkGenericInput(input_fn: FT, # FT Components that generates a user input (e.g. `TextArea`)
                    label:str|FT=(), # String or FT component that goes in `Label`
                    lbl_cls:str|Enum=(), # Additional classes that goes in `Label`
                    inp_cls:str|Enum=(), # Additional classes that go in user input (e.g. `TextArea`)
                    cls:str|Enum=('space-y-2',), # Div cls
                    id: str="", # ID of the user input (e.g. `TextArea`)
                   **kwargs # Passed to `input_fn` (e.g. ` TextArea`)
                  ) -> FT: # FT component in structure `(Div(label,input))`
    "`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UKTextArea` which is created for you in this library"
    lbl_cls, inp_cls, cls = map(stringify,(lbl_cls, inp_cls, cls))
    if label: 
        label = Label(cls='uk-form-label '+lbl_cls)(label)
        if id: label.attrs['for'] = id
    res = input_fn(**kwargs)
    if inp_cls: res.attrs['class'] += inp_cls
    if id: res.id = id
    return Div(cls=cls)(label, res)

In [91]:
show_doc(UkGenericInput)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/components.py#L102){target="_blank" style="float:right; font-size:smaller"}

### UkGenericInput

>      UkGenericInput (input_fn:fastcore.xml.FT, label:str|fastcore.xml.FT=(),
>                      lbl_cls:str|enum.Enum=(), inp_cls:str|enum.Enum=(),
>                      cls:str|enum.Enum=('space-y-2',), id:str='', **kwargs)

*`Div(Label,Input)` component with Uk styling injected appropriately. Generally you should higher level API, such as `UKTextArea` which is created for you in this library*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| input_fn | FT |  | FT Components that generates a user input (e.g. `TextArea`) |
| label | str \| fastcore.xml.FT | () | String or FT component that goes in `Label` |
| lbl_cls | str \| enum.Enum | () | Additional classes that goes in `Label` |
| inp_cls | str \| enum.Enum | () | Additional classes that go in user input (e.g. `TextArea`) |
| cls | str \| enum.Enum | ('space-y-2',) | Div cls |
| id | str |  | ID of the user input (e.g. `TextArea`) |
| kwargs |  |  |  |
| **Returns** | **FT** |  | **FT component in structure `(Div(label,input))`** |

In [92]:
#| export
UkInput =     partial(UkGenericInput, partial(Input, cls='uk-input'))
UkSwitch =    partial(UkGenericInput, partial(CheckboxX,    cls='uk-toggle-switch uk-toggle-switch-primary')) 
UkTextArea =  partial(UkGenericInput, partial(Textarea,     cls='uk-textarea'))
UkFormLabel = partial(UkGenericInput, partial(Uk_input_tag, cls='uk-form-label'))


In [97]:
show_doc(UkInput,title_level=4)

---

#### partial(<function ft_hx at 0x12ef8ed40>, 'input', cls='uk-input'))

>      partial(<function ft_hx at 0x12ef8ed40>, 'input', cls='uk-input'))
>              (label:str|fastcore.xml.FT=(), lbl_cls:str|enum.Enum=(),
>              inp_cls:str|enum.Enum=(), cls:str|enum.Enum=('space-y-2',),
>              id:str='', **kwargs)

*partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.*

In [102]:
UkInput("MyLabel",lbl_cls=TextT.muted_sm, inp_cls='font-bold',  id='myid')

```html
<div class="space-y-2">
<label class="uk-form-label text-sm uk-text-muted" for="myid">MyLabel</label>  <input class="uk-inputfont-bold" id="myid">
</div>

```

In [98]:
show_doc(UkSwitch,title_level=4)

---

#### partial(<function CheckboxX at 0x12ef8f6a0>, cls='uk-toggle-switch uk-toggle-switch-primary'))

>      partial(<function CheckboxX at 0x12ef8f6a0>, cls='uk-toggle-switch uk-
>              toggle-switch-primary')) (label=(), lbl_cls=(), inp_cls=(),
>              cls=('space-y-2',), id='', **kwargs)

*partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.*

In [99]:
show_doc(UkTextArea,title_level=4)

---

#### partial(<function ft_hx at 0x12ef8ed40>, 'textarea', cls='uk-textarea'))

>      partial(<function ft_hx at 0x12ef8ed40>, 'textarea', cls='uk-textarea'))
>              (label=(), lbl_cls=(), inp_cls=(), cls=('space-y-2',), id='',
>              **kwargs)

*partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.*

In [100]:
show_doc(UkFormLabel,title_level=4)

---

#### _f at 0x12ef825c0>, cls='uk-form-label'))

>      _f at 0x12ef825c0>, cls='uk-form-label')) (label=(), lbl_cls=(),
>                                                 inp_cls=(),
>                                                 cls=('space-y-2',), id='',
>                                                 **kwargs)

*partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.*

In [122]:
#| export
def Options(*c, # Content for an `Option`
            selected_idx=None # Index location of selected `Option`
           ):
    "Generates list of `Option`s with the proper `selected_idx`"
    return [Option(o,selected=i==selected_idx) for i,o in enumerate(c)]

In [123]:
show_doc(Options)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/components.py#L124){target="_blank" style="float:right; font-size:smaller"}

### Options

>      Options (*c, selected_idx=None)

*Generates list of `Option`s with the proper `selected_idx`*

In [124]:
_opts = Options('1',P('2'),H1('3'),selected_idx=1)
_opts

[option(('1',),{'selected': False}),
 option((p(('2',),{}),),{'selected': True}),
 option((h1(('3',),{}),),{'selected': False})]

In [125]:
#| export
def UkSelect(*c, # `Option` with content ready to be used,
            label=(), # String or FT component that goes in `Label`
            lbl_cls=(), # Additional classes that goes in `Label`
            inp_cls=(), # Additional classes that go in `Uk_select`
            cls=('space-y-2',), # Div cls
            id="", # ID of the `Uk_select`
            **kwargs # Passed to `Uk_select`
            ): # FT component in structure `(Div(label,`Uk_select`))`
    "`Div(Label,Uk_select)` component with Uk styling injected appropriately."
    lbl_cls, inp_cls, cls = map(stringify,(lbl_cls, inp_cls, cls))
    if label:
        label = Label(cls='uk-form-label' + lbl_cls)(label)
        if id: label.fr = id
    res = Uk_select(cls=inp_cls, uk_cloak=True, **kwargs)(*c)
    if id: res.id = id
    return Div(cls=cls)(label, res)

In [126]:
show_doc(UkSelect)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/components.py#L128){target="_blank" style="float:right; font-size:smaller"}

### UkSelect

>      UkSelect (*c, label=(), lbl_cls=(), inp_cls=(), cls=('space-y-2',),
>                id='', **kwargs)

*`Div(Label,Uk_select)` component with Uk styling injected appropriately.*

In [38]:
UkSelect(*_opts, "MyLabel",lbl_cls=TextT.muted_sm, inp_cls='font-bold',  id='myid')

```html
<div class="space-y-2">
<uk-select uk-cloak class="font-bold" id="myid"><optionoption>1</option><option selected>      <p>2</p>
</option><optionoption>      <h1>3</h1>
</option>MyLabel</uk-select></div>

```

In [40]:
#| export
class UkButtonT(VEnum):
    default = 'default'
    primary = 'primary'
    secondary = 'secondary'
    danger = 'danger'
    ghost = 'ghost'
    text = 'text'
    link = 'link'

In [43]:
#| export
def UkDropdownButton(label, # String or FT component that goes in the `Button`
                    options, # list of tuples that contain what you want listed (FT component/str)
                    btn_cls=UkButtonT.default, # Button class(es)
                    cls=(), # Parent div class
                    dd_cls=(), # Class that goes on each of the `options`
                    ): # FT component in structure `(Div(Div(Btn,dd)))`
    "`(Div(Div(Btn,dd)))` component with Uk styling injected appropiately to create a drop down effect"
    dd_cls, btn_cls, cls = map(stringify, (dd_cls, btn_cls, cls))
    btn = Button(type='button', cls='uk-button ' + btn_cls)(label, Span(uk_icon='icon: triangle-down'))
    dd_opts = [Li(A(href="#demo", cls=f'uk-drop-close w-full ' + dd_cls, uk_toggle=True, role="button")(o) for o in options)]
    dd = Div(uk_drop='mode: click; pos: bottom-right', cls='uk-dropdown uk-drop')(Ul(cls='uk-dropdown-nav')(*([Li(cls='uk-nav-divider')] + dd_opts)))
    return Div(cls=cls)(Div(cls='flex items-center space-x-4')(btn, dd))

In [44]:
show_doc(UkDropdownButton)

---

[source](https://github.com/Isaac-Flath/fh-frankenui/blob/main/fh_frankenui/components.py#L153){target="_blank" style="float:right; font-size:smaller"}

### UkDropdownButton

>      UkDropdownButton (label, options, btn_cls=<UkButtonT.default: 'default'>,
>                        cls=(), dd_cls=())

*`(Div(Div(Btn,dd)))` component with Uk styling injected appropiately to create a drop down effect*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| label |  |  | String or FT component that goes in the `Button` |
| options |  |  | list of tuples that contain what you want listed (FT component/str) |
| btn_cls | UkButtonT | uk-button-default | Button class(es) |
| cls | tuple | () | Parent div class |
| dd_cls | tuple | () | Class that goes on each of the `options` |

### Other

In [17]:
#| export
def UkButton(*c, 
            cls=UkButtonT.default, # Use UkButtonT or styles 
            **kwargs):    
    return Button(type='button', cls='uk-button ' + stringify(cls), **kwargs)(*c)

In [18]:
from fasthtml.common import *
Div(P(),Button())

```html
<div>
  <p></p>
<button></button></div>

```

In [19]:
#| export
def UkGenericComponent(component_fn, *c, cls=(), **kwargs):
        res = component_fn(**kwargs)(*c)
        if cls: res.attrs['class'] += ' ' + cls
        return res

UkH1 = partial(UkGenericComponent, partial(H1,cls='uk-h1'))
UkH2 = partial(UkGenericComponent, partial(H2,cls='uk-h2'))
UkH3 = partial(UkGenericComponent, partial(H3,cls='uk-h3'))
UkH4 = partial(UkGenericComponent, partial(H4,cls='uk-h4'))
UkH5 = partial(UkGenericComponent, partial(H5,cls='uk-h5'))
UkH6 = partial(UkGenericComponent, partial(H6,cls='uk-h6'))


In [20]:
UkH1("My Uk styled header"), UkH1('My Uk styled header',cls='another-class')

(h1(('My Uk styled header',),{'class': 'uk-h1'}),
 h1(('My Uk styled header',),{'class': 'uk-h1 another-class'}))

In [21]:
#| export
def UkHSplit(*c, cls=(), line_cls=(), text_cls=()):
    cls, line_cls, text_cls = map(stringify,(cls, line_cls, text_cls))
    return Div(cls='relative ' + cls)(
        Div(cls="absolute inset-0 flex items-center " + line_cls)(Span(cls="w-full border-t border-border")),
        Div(cls="relative flex justify-center " + text_cls)(Span(cls="bg-background px-2 ")(*c)))

## Higher order components

In [22]:
#|export
def Card(*c, # Components that go in the body
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls=(), # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [Div(cls='uk-card-header ' + header_cls)(header),]
    res += [Div(cls='uk-card-body ' + body_cls)(*c),]
    if footer: res += [Div(cls='uk-card-footer ' + footer_cls)(footer),]
    return Div(cls='uk-card '+cls, **kwargs)(*res)

In [23]:
#| export
def UkModalTitle(*c, cls=()): return Div(cls='uk-modal-title ' + stringify(cls))(*c)

def Modal(*c,
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls='p-6', # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [Div(cls='uk-modal-header ' + header_cls)(header),]
    res += [Div(cls='uk-modal-body uk-modal-dialog ' + body_cls)(*c),]
    if footer: res += [Div(cls='uk-modal-footer ' + footer_cls)(footer),]
    return Div(cls='uk-modal uk-modal-container' + cls, uk_modal=True, **kwargs)(*res)


In [25]:
#| hide


import nbdev; nbdev.nbdev_export()